In [69]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import contractions
#import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import Perceptron

import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 


nltk.download('punkt', quiet = True)
nltk.download('wordnet', quiet = True)
nltk.download('averaged_perceptron_tagger', quiet = True)
nltk.download('stopwords', quiet = True)
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
! pip install zstandard

     -------------------------------------- 473.8/473.8 kB 7.5 MB/s eta 0:00:00


In [12]:
## using in block processing to make data visualize easier
# def preprocess_data(df):
#     # lower case
#     df['Text'] = df['Text'].str.lower()

#     # remove url
#     df['Text'] = df['Text'].replace(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?', 'URL', regex=True)

#     # remove non-alphabetical
#     df['Text'] = df['Text'].str.replace('[^a-zA-Z\']', ' ', regex=True).str.strip()


#     # remove extra spaces
#     df['Text'] = df['Text'].str.replace(' +', ' ', regex=True).str.strip()

#     # #expand contractions
#     df['Text'] = df['Text'].apply(lambda x: contractions.fix(x))
#     # df['Text'] = df['Text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
#     # #join back words
#     # df['Text'] = [' '.join(map(str, l)) for l in X['Text']]

#     # #remove html and url form reviews
#     # df['Text'] = df['Text'].str.replace(r'\s*https?://\S+(\s+|$)', '', regex=True).str.strip()
#     # #remove non-alphabetical characters
#     # df['Text'] = df['Text'].str.replace('[^a-zA-Z]', ' ', regex=True)
#     # #remove extra spaces
#     # df['Text'] = df['Text'].replace(r'\s+', ' ', regex=True)
#     return df

In [13]:
def tokenize_and_remove_stopwords(X):
    stop_words = set(stopwords.words('english'))
    X['tokens'] = X['Text'].apply(tokenize.word_tokenize)
    return X['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [14]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [15]:
def vectorize_tfidf(X):
    tfidf=TfidfVectorizer(min_df = 50, max_df=0.95, ngram_range = (1,3), max_features=1500, norm='l2')
    X_data = tfidf.fit_transform(X)
    return X_data.toarray()

In [62]:
#read csv file and convert to dataframe
df = pd.read_pickle('data/2019.pkl.zst',compression = 'zstd')

In [63]:
 # remnove nan from the df
df = df.dropna(how="any")

In [64]:
df

,Title,Docket,Argued,Decided,Text
0,AARON J. SCHOCK v. UNITED STATES(2019),No. 18-406,,"February 19, 2019",United States Supreme Court AARON J. SCHOCK v...
1,"ABU-ALI ABDUR'RAHMAN, ET AL. v. TONY PARKER,...",No. 18-8332,,"May 13, 2019",United States Supreme Court ABU-ALI ABDUR'RAHM...
2,AIR &AMP; LIQUID SYSTEMS CORP. ET AL. v. DEVRI...,No. 17-1104,"October 10, 2018","March 19, 2019",United States Supreme Court AIR &AMP; LIQUID S...
3,AMERICAN LEGION ET AL. v. AMERICAN HUMANIST AS...,No. 17-1717,"February 27, 2019","June 20, 2019",United States Supreme Court AMERICAN LEGION ET...
4,APPLE INC. v. PEPPER ET AL.(2019),No. 17-204,"November 26, 2018","May 13, 2019",United States Supreme Court APPLE INC. v. PEPP...
...,...,...,...,...,...
62,TIMBS v. INDIANA(2019),No. 17-1091,"November 28, 2018","February 20, 2019",United States Supreme Court TIMBS v. INDIANA(2...
63,VIRGINIA HOUSE OF DELEGATES ET AL. v. BETHUNE-...,No. 18-281,"March 18, 2019","June 17, 2019",United States Supreme Court VIRGINIA HOUSE OF ...
64,"VIRGINIA URANIUM, INC., ET AL. v. WARREN ET AL...",No. 16-1275,"November 5, 2018","June 17, 2019","United States Supreme Court VIRGINIA URANIUM, ..."
65,"WALTER DANIEL, INDIVIDUALLY AND AS PERSONAL RE...",No. 18-460,,"May 20, 2019","United States Supreme Court WALTER DANIEL, IND..."


In [65]:
df['Text'].head()[0]

'United States Supreme Court AARON J. SCHOCK  v.  UNITED STATES(2019) No. 18-406 Argued: Decided: February 19, 2019         The petition for a writ of certiorari is denied.      Statement of Justice Sotomayor respecting the denial of certiorari.      Petitioner Aaron Schock, a former Congressman from Illinois, asks us to decide whether he may immediately appeal, as a collateral order, the denial of his motion to dismiss part of a criminal indictment against him for running afoul of the Constitution\'s Rulemaking Clause.  See Art. I, S5.  He argues that certain charges against him would require the District Court for the Central District of Illinois to interpret internal rules adopted by the House of Representatives to govern its own Members, and thus would violate separation-of-powers doctrine.  The Court of Appeals for the Seventh Circuit held that denials of such Rulemaking Clause challenges are not collateral orders subject to immediate appeal, 891 F. 3d 334 (2018), in disagreement 

In [66]:
# Disabled as it might be important later on
# # lower case
# df['Text'] = df['Text'].str.lower()

# remove url
df['Text'] = df['Text'].str.replace(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?', 'URL', regex=True)

# Disabled as legal cases need punctuations to work
# # remove non-alphabetical
# df['Text'] = df['Text'].str.replace('[^a-zA-Z0-9\'\".!()]', ' ', regex=True).str.strip()


# remove extra spaces
df['Text'] = df['Text'].str.replace(' +', ' ', regex=True).str.strip()

# #expand contractions
df['Text'] = df['Text'].apply(lambda x: contractions.fix(x))
df['Text'].head()[0]

'United States Supreme Court AARON J. SCHOCK v. UNITED STATES(2019) No. 18-406 Argued: Decided: February 19, 2019 The petition for a writ of certiorari is denied. Statement of Justice Sotomayor respecting the denial of certiorari. Petitioner Aaron Schock, a former Congressman from Illinois, asks us to decide whether he may immediately appeal, as a collateral order, the denial of his motion to dismiss part of a criminal indictment against him for running afoul of the Constitution\'s Rulemaking Clause. See Art. I, S5. He argues that certain charges against him would require the District Court for the Central District of Illinois to interpret internal rules adopted by the House of Representatives to govern its own Members, and thus would violate separation-of-powers doctrine. The Court of Appeals for the Seventh Circuit held that denials of such Rulemaking Clause challenges are not collateral orders subject to immediate appeal, 891 F. 3d 334 (2018), in disagreement with at least one other

# Establish base line performance

In [17]:
# # preprocess df['tokens'] using preocess_text function
# data = df[:100]


In [40]:
# df['tokens'] = preprocess_data(data)


In [67]:
# df['tokens'] = tokenize_and_remove_stopwords(df['tokens'])

In [70]:
stop_words = set(stopwords.words('english'))
df['tokens'] = df['Text'].apply(tokenize.word_tokenize)
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df['tokens'][0]

['United',
 'States',
 'Supreme',
 'Court',
 'AARON',
 'J.',
 'SCHOCK',
 'v.',
 'UNITED',
 'STATES',
 '(',
 '2019',
 ')',
 'No',
 '.',
 '18-406',
 'Argued',
 ':',
 'Decided',
 ':',
 'February',
 '19',
 ',',
 '2019',
 'The',
 'petition',
 'writ',
 'certiorari',
 'denied',
 '.',
 'Statement',
 'Justice',
 'Sotomayor',
 'respecting',
 'denial',
 'certiorari',
 '.',
 'Petitioner',
 'Aaron',
 'Schock',
 ',',
 'former',
 'Congressman',
 'Illinois',
 ',',
 'asks',
 'us',
 'decide',
 'whether',
 'may',
 'immediately',
 'appeal',
 ',',
 'collateral',
 'order',
 ',',
 'denial',
 'motion',
 'dismiss',
 'part',
 'criminal',
 'indictment',
 'running',
 'afoul',
 'Constitution',
 "'s",
 'Rulemaking',
 'Clause',
 '.',
 'See',
 'Art',
 '.',
 'I',
 ',',
 'S5',
 '.',
 'He',
 'argues',
 'certain',
 'charges',
 'would',
 'require',
 'District',
 'Court',
 'Central',
 'District',
 'Illinois',
 'interpret',
 'internal',
 'rules',
 'adopted',
 'House',
 'Representatives',
 'govern',
 'Members',
 ',',
 'thus'

In [71]:
df['tokens'] = df['tokens'].apply(nltk.tag.pos_tag)


0     [(United, NNP), (States, NNPS), (Supreme, NNP)...
1     [(United, NNP), (States, NNPS), (Supreme, NNP)...
2     [(United, NNP), (States, NNPS), (Supreme, NNP)...
3     [(United, NNP), (States, NNPS), (Supreme, NNP)...
4     [(United, NNP), (States, NNPS), (Supreme, NNP)...
                            ...                        
62    [(United, NNP), (States, NNPS), (Supreme, NNP)...
63    [(United, NNP), (States, NNPS), (Supreme, NNP)...
64    [(United, NNP), (States, NNPS), (Supreme, NNP)...
65    [(United, NNP), (States, NNPS), (Supreme, NNP)...
66    [(United, NNP), (States, NNPS), (Supreme, NNP)...
Name: tokens, Length: 67, dtype: object

In [21]:
df['tokens'] = df['tokens'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [22]:
lemma = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda x: [lemma.lemmatize(word, tag) for word, tag in x])

In [23]:
df['tokens'] = [' '.join(map(str, l)) for l in df['tokens']]

In [24]:
#vectorize the text using tfidf vectorizer
X_data = vectorize_tfidf(df['tokens'])

In [25]:
X_data.shape

(100, 434)

In [26]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [27]:
w2v_model = Word2Vec(df['tokens'], vector_size=300, window=5, min_count=10)

In [28]:
def create_word2vec_embeddings(X):
    embeddings = []
    for sentence in X:
        sentence_embedding = np.zeros(300)
        for word in sentence:
            if word in w2v_model.wv:
                sentence_embedding += w2v_model.wv[word]
        embeddings.append(sentence_embedding)
    return np.array(embeddings)

In [29]:
w2v_vectors = create_word2vec_embeddings(df['tokens'])

In [30]:
w2v_vectors.shape

(100, 300)

## TF-IDF Model

In [31]:
import gensim
import gensim.downloader as api

def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

def doc2vec_vectors(X):
    documents = list(tagged_document(X[0].split()))
    model = gensim.models.doc2vec.Doc2Vec(documents, vector_size=300, window=5, min_count=10)
    return np.array([model.infer_vector(doc.split()) for doc in X])

In [32]:
doc2vec = doc2vec_vectors(df['tokens'])

In [33]:
#find most similar cases using cosine similarity and tfidf vectors
def find_similar_cases(X, case_id, n):
    pairwise_similarities=np.dot(X,X.T)
    most_similar = pairwise_similarities[case_id].argsort()[:-n-1:-1]
    return most_similar

#find most similar cases using cosine similarity and word2vec vectors

find_similar_cases(X_data, 0, 4)


array([0, 5, 4, 8], dtype=int64)

## Word2Vec Model

In [34]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
#find the max sequence length of word2vec vectors\
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['tokens'])
tokenized_documents=tokenizer.texts_to_sequences(df['tokens'])
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=5000,padding='post')
vocab_size=len(tokenizer.word_index)+1


(5000,)


In [ ]:
w2v_vectors = create_word2vec_embeddings(df['tokens'])

## BERT Model

In [41]:
! pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(df['tokens'])

     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 5.5/5.5 MB 8.8 MB/s eta 0:00:00
     -------------------------------------- 167.2/167.2 MB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 11.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
     -------------------------------------- 151.6/151.6 kB 8.8 MB/s eta 0:00:00
  Using cached tokenizers-0.13.1-cp39-cp39-win_amd64.whl (3.3 MB)
     ---------------------------------------- 2.5/2.5 MB 9.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125930 sha256=695baba5210bbbaf5df1b9d4bb5ab3754c05bb66f78afacf770b8cb2529f2050
  Stored in directory: c:\users\a

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

similar = cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[:]
)
ind = np.argsort(similar[0])[::-1][:5]

In [43]:
ind

array([ 0, 98,  2,  7, 88], dtype=int64)